In [4]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from time import time
import re

n_features = 10000
n_components = 10
n_top_word = 7

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

print("Loading dataset...")

dataset = pd.read_csv('https://raw.githubusercontent.com/Matheusadler/Sentiment-Analysis-Application/master/CSV%20Cleaner/g1_clean%20CLA.csv', sep=',', encoding='UTF-8')

data_samples = dataset.iloc[1:,0]

n_samples = len(data_samples)

#Loading StopWords
from io import open
with open("preprocessamento\essenciais\stopwords_pt_nltk.txt","r",encoding='utf-8') as a:
    StopWords = a.readlines()
    StopWords = [w.replace('\n', '') for w in StopWords]


print("Dataset loaded!")

clean_comments = []
for w in range(len(dataset.Comment)):
  comment = dataset['Comment'].iloc[w]

  # remove special characters and digits
  comment  = re.sub("(\\d|\\W)+|\w*\d\w*"," ",comment )
  comment = ' '.join(s for s in comment.split() if (not any(c.isdigit() for c in s)) and len(s) > 2)
  clean_comments.append(comment)

clean_comments[0:5]

print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words=StopWords)
tfidf = tfidf_vectorizer.fit_transform(clean_comments)

print("tf-idf features extracted!")

# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words=StopWords)
tf = tf_vectorizer.fit_transform(clean_comments)

print("tf features for LDA extraction is completed!")

# Fit the NMF model
print("Fitting the NMF model (Frobenius norm) with tf-idf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          alpha=.1, l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model (Frobenius norm):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_word)

# Fit the NMF model
print("Fitting the NMF model (generalized Kullback-Leibler divergence) with "
      "tf-idf features, n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model (generalized Kullback-Leibler divergence):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_word)

print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_word)

print("Fiting LSA model")

lsa = TruncatedSVD(n_components=n_components, n_iter=40, tol=0.01)

lsa.fit(tf)

print("\nTopics in LSA model:")

print_top_words(lsa, tf_feature_names, n_top_word)

e:\python\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:31: DeprecationWarning: invalid escape sequence \e
<>:43: DeprecationWarning: invalid escape sequence \w
<>:31: DeprecationWarning: invalid escape sequence \e
<>:43: DeprecationWarning: invalid escape sequence \w
<>:31: DeprecationWarning: invalid escape sequence \e
<>:43: DeprecationWarning: invalid escape sequence \w
<ipython-input-4-d7b4e65077b1>:31: DeprecationWarning: invalid escape sequence \e
  with open("preprocessamento\essenciais\stopwords_pt_nltk.txt","r",encoding='utf-8') as a:
<ipython-input-4-d7b4e65077b1>:43: DeprecationWarning: invalid escape sequence \w
  comment  = re.sub("(\\d|\\W)+|\w*\d\w*"," ",comment )


Loading dataset...
Dataset loaded!
Extracting tf-idf features for NMF...
tf-idf features extracted!
Extracting tf features for LDA...
tf features for LDA extraction is completed!
Fitting the NMF model (Frobenius norm) with tf-idf features, n_samples=186 and n_features=10000...
done in 0.052s.

Topics in NMF model (Frobenius norm):
Topic #0: vai corolla povo carro coisa ainda gente
Topic #1: bmw cla motor mercedes preço opinião demais
Topic #2: mundo classe total fiasco estrelinha resultado mercedes
Topic #3: tração audi dianteira melhor ainda traseira gente
Topic #4: anda etc acho fusion boa azera conjunto
Topic #5: cla chega versão meio mercedes alto vai
Topic #6: anos ocorrida tragédia após cla lançado lançamento
Topic #7: mil preço comprar melhor deve caro mercedes
Topic #8: lucro querem imposto porte ganhar custo alem
Topic #9: carro mercedes desse igual carros custa todo

Fitting the NMF model (generalized Kullback-Leibler divergence) with tf-idf features, n_samples=186 and n_feat

e:\python\lib\site-packages\sklearn\feature_extraction\text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['comprei', 'demora', 'falta', 'fs', 'nã', 'reclame', 'sercomtel', 'vas', 'veiculo', 'vo'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


done in 0.190s.

Topics in NMF model (generalized Kullback-Leibler divergence):
Topic #0: vai carro custa povo pior governo coisa
Topic #1: bmw caro preço cla motor mercedes todos
Topic #2: mundo todo classe mercedes ver menos fiasco
Topic #3: audi tração ainda dianteira gente muita melhor
Topic #4: anda conjunto frente opinião mecânica ponto carros
Topic #5: verdade cla falem difícil chega boa meio
Topic #6: cla anos lançado nunca agora ocorrida lançamento
Topic #7: mil comprar preço melhor preços vou uns
Topic #8: civic querem lucro ganhar compra custo comprar
Topic #9: carro mercedes desse carros etc pobre igual

Fitting LDA models with tf features, n_samples=186 and n_features=10000...
done in 0.258s.

Topics in LDA model:
Topic #0: comprar dinheiro cadenza azera motor equipado ambos
Topic #1: povo civic governo coisa difícil ver gustavo
Topic #2: carro igual mercedes desse mim benz pro
Topic #3: carro mil tração comprar deve preços devia
Topic #4: corolla lucro carro vai grande co

In [7]:
pyLDAvis.sklearn.prepare(lda, tfidf, tf_vectorizer, sort_topics=False, mds = 'tsne')

e:\python\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
0      32.420952   8.115230       1        1   2.407782
1       0.648955  71.707100       2        1   6.225535
2      71.019684   9.971006       3        1  16.153891
3       3.785981  33.185135       4        1  12.621594
4     -42.666500  -9.222070       5        1   9.885571
5      -3.621689 -43.526516       6        1   5.014156
6      -3.632402  -4.171708       7        1  14.223284
7     -36.584400  35.804413       8        1  14.870250
8      44.391380  50.615383       9        1   7.976454
9      40.235809 -31.591665      10        1  10.621484, topic_info=       Term       Freq      Total Category  logprob  loglift
74    carro  18.000000  18.000000  Default  30.0000  30.0000
408    povo   4.000000   4.000000  Default  29.0000  29.0000
390    pior   4.000000   4.000000  Default  28.0000  28.0000
326     mil   8.000000   8.000000  Default  27.0000  27.0000
58      bmw   5.000000   5.000000  Default  26.0000  26.0000
..      ...        ...        ...      ...      ...      ...
91    civic   1.124497   6.157668  Topic10  -5.1535   0.5419
93   classe   0.913738   5.111212  Topic10  -5.3611   0.5206
411   preço   0.917741   8.799860  Topic10  -5.3567  -0.0183
74    carro   0.920967  18.966785  Topic10  -5.3532  -0.7827
59      boa   0.822520   3.854243  Topic10  -5.4663   0.6977

[436 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
0         8  0.344930   abaixo
2         3  0.308846  absurdo
5         7  0.309991   accord
6         7  0.301959   acesso
8         5  0.312866    achar
...     ...       ...      ...
535       3  0.212168      vou
535       7  0.424335      vou
537       5  0.325312      xei
540      10  0.318802    ítens
542       5  0.346608   ônibus

[439 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])